In [ ]:
%pip install fsspec gcsfs

In [1]:
import duckdb
import os
from fsspec import filesystem



In [2]:
cn = duckdb.connect()
cn.execute("INSTALL iceberg; INSTALL httpfs;")
cn.execute("LOAD iceberg;")
cn.register_filesystem(filesystem('gcs'))
endpoint = "https://biglake.googleapis.com/iceberg/v1beta/restcatalog"

/Users/matthewmartin/dream_machine/duckdb/iceberg/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
import gcs_auth
gcs = gcs_auth.BigLakeAuthHelper(cn)
token = gcs.get_token()

/Users/matthewmartin/dream_machine/duckdb/iceberg/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
cn.execute(f"""
    CREATE or replace SECRET gcs_auth (
        TYPE gcs,
        KEY_ID '{os.getenv("GCS_KEY")}',
        SECRET '{os.getenv("GCS_SECRET")}'
    );        
""")

In [5]:


bucket = os.getenv("GCS_BUCKET")

print(bucket)

data-mattm-test-sbx


In [8]:
cn.sql(f"""
   select *
       from read_parquet('gcs://{bucket}/my_new_flock/*.parquet')    
""")

┌───────┬─────────┐
│  id   │  name   │
│ int32 │ varchar │
├───────┼─────────┤
│     1 │ Alice   │
│     2 │ Bob     │
└───────┴─────────┘

In [ ]:

# wonder if its a duckdb thing or something els
#/ this is the spark example google has: https://cloud.google.com/bigquery/docs/blms-rest-catalog#required_roles
# maybe try it first?
cn.execute(f"""
    ATTACH 'gcs://{bucket}/ice_ice_baby' 
    AS biglake
        (
            TYPE ICEBERG,
            ENDPOINT '{endpoint}',
            AUTHORIZATION_TYPE oauth2,
            token '{token}'
        )
""")


Error: Request to 'https://biglake.googleapis.com/iceberg/v1beta/restcatalog/v1/config?warehouse=gcs%3A%2F%2Fdata-mattm-test-sbx%2Fice_ice_baby' returned a non-200 status code (Forbidden_403), with reason: Forbidden

In [ ]:
#CREATE TABLE biglake.`myproject.mydataset`.duckdb_test (...)